In [1]:
]st

    Status `/mnt/E4E0A9C0E0A998F6/github/ReinforcementLearningAnIntroduction.jl/notebooks/Project.toml`
  [02c1da58] RLIntro v0.2.0 [`..`]
  [158674fc] ReinforcementLearning v0.4.0 [`../../ReinforcementLearning.jl`]
  [25e41dd2] ReinforcementLearningEnvironments v0.1.1


In [3]:
using ReinforcementLearning, ReinforcementLearningEnvironments, RLIntro.MultiArmBandits

UndefVarError: UndefVarError: TenArmedTestBed not defined